In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
import tqdm
import talib as ta
import mplfinance as mpf
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import shutil
import math

In [2]:
stock_basic_df = pd.read_csv("./data/stock_basic_df.csv")
ggt_df = pd.read_csv("./data/ggt_basic_df.csv")
etf_df = pd.read_csv("./data/etf_basic_df.csv")
fetf_df = pd.read_csv("./data/fetf_basic_df.csv")
basic_df = pd.read_csv("./data/basic_df.csv")

def chnname(code):
    name = ''
    if code in basic_df['ts_code'].values:
        name = basic_df.loc[basic_df['ts_code'] == code, 'name'].values[0]
    elif code in fetf_df['ts_code'].values:
        name = fetf_df.loc[fetf_df['ts_code'] == code, 'name'].values[0]
    return name
def chnindustry(code):
    industry = ''
    if code in basic_df['ts_code'].values:
        industry = basic_df.loc[basic_df['ts_code'] == code, 'industry'].values[0]
    return industry


In [3]:
industry_df = pd.read_csv("./data/industry.csv")
buy_list = pd.read_csv("./data/buy_list.csv")['Column1'].tolist()
up_list = pd.read_csv("./data/up_list.csv")['Column1'].tolist()
daily_data_dir = './data/daily'


In [4]:
def apply_score(df,period=25):
    df.sort_values(by="trade_date", ascending=True, inplace=True)
    df = df.tail(period)
    y = np.log(df.close)
    x = np.arange(len(y))
    slope, intercept = np.polyfit(x, y, 1)
    annualized_returns = math.pow(math.exp(slope), 250) - 1
    r_squared = 1 - (sum((y - (slope * x + intercept))**2) / ((len(y) - 1) * np.var(y, ddof=1)))
    score = annualized_returns * r_squared
    return score,annualized_returns,r_squared

def apply_bbands(df, period=20):
    df.sort_values(by="trade_date", ascending=True, inplace=True)
    upper, middle, lower = ta.BBANDS(df['close'], timeperiod=period, nbdevup=2, nbdevdn=2, matype=0)
    df['rel'] = (df['close'] - lower) / (upper - lower)
    return df


def apply_rsi(df, period=24):
    df = df.sort_values(by="trade_date", ascending=True)
    df["rsi12"] = ta.RSI(df["close"].values, timeperiod=12)
    df["rsi60"] = ta.RSI(df["close"].values, timeperiod=60)
    df["rsi120"] = ta.RSI(df["close"].values, timeperiod=120)
    df['min_rsi'] = df[['rsi12', 'rsi60', 'rsi120']].min(axis=1)

    return df

In [5]:
# 计算每个组的比例
close_greater_than_ma60 = industry_df['close'] > industry_df['ma60']
close_greater_than_ma20 = industry_df['close'] > industry_df['ma20']
close_greater_than_ma5 = industry_df['close'] > industry_df['ma5']

industry_proportion_ma60 = close_greater_than_ma60.groupby(industry_df['industry']).mean() * 100
industry_proportion_ma20 = close_greater_than_ma20.groupby(industry_df['industry']).mean() * 100
industry_proportion_ma5 = close_greater_than_ma5.groupby(industry_df['industry']).mean() * 100

# 创建包含比例的 DataFrame
industry_proportion = pd.DataFrame({
    'MA5 Proportion': round(industry_proportion_ma5, 2),
    'MA20 Proportion': round(industry_proportion_ma20, 2),
    'MA60 Proportion': round(industry_proportion_ma60, 2),
})
industry_proportion.sort_values(by='MA20 Proportion', ascending=False, inplace=True)

# 计算每个组的行数
industry_counts = close_greater_than_ma5.groupby(industry_df['industry']).size().reset_index(name='MA5 Count')

# 将比例和行数合并到一个 DataFrame 中
industry_proportion_with_counts = pd.merge(industry_proportion, industry_counts, on='industry')

# 设置显示选项以显示所有行
pd.set_option('display.max_rows', None)

# 输出结果
industry_proportion_with_counts

,industry,MA5 Proportion,MA20 Proportion,MA60 Proportion,MA5 Count
0,超市连锁,11.11,100.00,100.00,9
1,旅游景点,0.00,94.12,94.12,17
2,酒店餐饮,0.00,88.89,88.89,9
3,百货,14.29,88.57,100.00,35
4,火力发电,68.75,87.50,81.25,32
5,啤酒,12.50,87.50,100.00,8
6,银行,52.38,73.81,95.24,42
7,影视音像,11.43,68.57,100.00,35
8,路桥,52.63,68.42,89.47,19
9,船舶,50.00,66.67,50.00,12


In [6]:
industry_list = ['银行','普钢','煤炭开采','小金属','焦炭加工','火力发电']
industry_proportion_with_counts[industry_proportion_with_counts['industry'].isin(industry_list)]

,industry,MA5 Proportion,MA20 Proportion,MA60 Proportion,MA5 Count
4,火力发电,68.75,87.50,81.25,32
6,银行,52.38,73.81,95.24,42
22,普钢,26.92,53.85,88.46,26
42,煤炭开采,32.00,44.00,48.00,25
53,焦炭加工,12.50,37.50,87.50,8
98,小金属,9.09,20.00,52.73,55


In [7]:
# hold and recent sell
hold_dict = {
    '000028.SZ': 29.78, # 国药一致
    '000157.SZ': 7.00, # 中联重科
    '000526.SZ': 47.93, # 学大
    '000537.SZ': 9.45, # 中绿电
    '001234.SZ': 16.25, # 泰慕士
    '001260.SZ': 11.88, # 坤泰
    '001299.SZ': 13.73, # 美能能源
    '001366.SZ': 11.64, # 播恩
    '001376.SZ': 15.8, # 百通
    '001979.SZ': 11.65, # 招商蛇口
    '002546.SZ': 4.23, # 新联电子
    '159529.SZ': 1.404, # 标普消费
    '159869.SZ': 1.064, # 游戏 etf
    '159941.SZ': 1.203, # 纳指
    '300784.SZ': 68.90, # 利安
    '300998.SZ': 17.20, # 宁波方正
    '301107.SZ': 28.84, # 瑜欣电子
    '301212.SZ': 26.35, # 联盛化学
    '511090.SH': 118.687, # 30年国债
    '511380.SH': 11.703, # 可转债
    '511520.SH': 112.668, # 政金债
    '513030.SH': 1.487, # 德国
    '513400.SH': 1.197, # 道琼斯
    '513500.SH': 2.23, # 标普500
    '513730.SH': 1.412, # 东南亚
    '515790.SH': 0.894, # 光伏
    '516160.SH': 2.062, # 新能源
    '562800.SH': 0.562, # 稀有金属
    '588290.SH': 1.501, # 芯片科创
    '600016.SH': 4.09, # 民生银行
    '600096.SH': 22.09, # 云天化
    '600325.SH': 6.61, # 华发股份
    '600373.SH': 13.51, # 中文传媒
    '600585.SH': 26.25, # 海螺水泥
    '600780.SH': 6.00, # 通宝能源
    '600900.SH': 28.39, # 长江电力
    '601068.SH': 4.75, # 中铝
    '601101.SH': 8.9, # 昊华科技
    '601117.SH': 8.38, # 中国化学
    '601666.SH': 10.13, # 平煤股份
    '603056.SH': 14.4, # 德邦股份
    '603167.SH': 8.71, # 渤海轮渡
    '603172.SH': 14.81, # 万丰
    '603588.SH': 5.4, # 高能环境
    '688036.SH': 96.98, # 传音控股
    '688443.SH': 30.56, # 智翔金泰
    '688530.SH': 21.67, # 欧莱
    '688638.SH': 30.07, # 誉辰智能
    '688695.SH': 34.91, # 中创
}
recent_sell_dict = {
    #12.12
    '601107.SH': 5.27, # 四川成渝

    #12.10
    '300995.SZ': 19.47, # 奇德新材
    '600036.SH': 38.92, # 招商银行
    '600064.SH': 8.33, # 南京高科
    #12.9
    '000543.SZ': 8.36, # 皖能电力
    '002508.SZ': 23.01, # 老板电器
    '600328.SH': 8.49, # 中盐化工
    #12.6
    '603280.SH':23.71, # 南方路机
    '560010.SH':2.58, # 1000
    #12.5
    '601669.SH': 5.5, # 中国电建
    #12.4
    '600089.SH':13.05, # 特变电工
    '601077.SH':5.69, # 渝农商行
    '002394.SH':8.21, # 联发股份
    # 12.3
    '600894.SH': 13.36, # 广日股份
    '002233.SZ': 7.71, # 塔牌
    '002191.SZ': 4.67, # 劲嘉
    '000517.SZ': 2.55, # 荣安
    '000504.SZ': 10.39, # 南华
    # 12.2
    '600153.SH': 10.8, # 建发
    # 11.28
    '600162.SH': 1.9 , # 香江
    # 11.27
    '515220.SH':1.166, # 煤炭
    # 11.26
    '601006.SH':6.66, # 大秦铁路
    # 11.25
    '513090.SH':1.479, # 香港证券
    '603816.SH':28.06, # 顾家家居
    '600008.SH':3.25 , # 首创
    '301359.SH':23.35 , # 东南
    '159922.SZ':6.065 , # 500
    '512200.SH':1.569 , # 地产
    '512480.SH':0.961 , # 半导体
    # 11.21
    '001256.SZ': 18.28, # 炜冈
    '001259.SZ': 24.02, # 利仁
    '601069.SH': 12.64, # 西部黄金
    # 11.20
    '603272.SH': 13.93, # 联翔
    '301234.SZ': 27.7, # 五洲医疗
    '001209.SZ': 14.22, # 洪兴
    '301309.SZ': 27.85, # 万德凯
    '001226.SZ': 25.56, # 拓山
    '301061.SZ': 55.61, # 匠心家居
    '000581.SZ': 18.29, # 威孚高科
    '603048.SH': 14.96, # 浙江黎明
    '688750.SH': 29.85, # 金天
    '002895.SZ': 21.4, # 川恒股份
    # 11.19
    '301043.SZ': 27.8, # 绿岛风
    '301388.SZ': 22.58, # 欣灵电气
    '001368.SZ': 19.55, # 通达创智
    '603216.SH': 10.49, # 梦天家居
    '603307.SH': 31.25, # 金泉
    '601919.SH': 14.6, # 中远海控
}

def apply_info(df):
    df['state'] = ''
    df['per'] = ''
    for i in df.index:
        ts_code = df.loc[i, 'ts_code']
        if ts_code in hold_dict:
            df.loc[i, 'state'] = 'hold'
            daily_df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
            daily_df = daily_df.sort_values(by="trade_date", ascending=True)
            close = daily_df.iloc[-1]['close']
            df.loc[i, 'per'] = round((close - hold_dict[ts_code]) / hold_dict[ts_code] * 100,1)
        elif ts_code in recent_sell_dict:
            df.loc[i, 'state'] = 'sell'
            daily_df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
            daily_df = daily_df.sort_values(by="trade_date", ascending=True)
            close = daily_df.iloc[-1]['close']
            df.loc[i, 'per'] = round((close - recent_sell_dict[ts_code]) / recent_sell_dict[ts_code] * 100,1)
            


In [19]:
mini_df = industry_df[industry_df['pe'] > 0]
mini_df = mini_df[mini_df['ts_code'].isin(up_list)]   

print(len(mini_df))
# base_mv_per = industry_df["circ_mv"].quantile(0.05)
# mini_df = mini_df[mini_df["circ_mv"] < base_mv_per]
#print(base_mv_per)
mini_df['circ_mv'] = round(mini_df['circ_mv'] / 10000,2)

for i in tqdm.tqdm(mini_df.index, desc="Processing"):
    ts_code = mini_df.loc[i, 'ts_code']
    df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
    #df = apply_bbands(df,period=60)
    score,annualized_returns,r_squared = apply_score(df,120)
    df = df.sort_values(by="trade_date", ascending=False)
    vol = df.iloc[0]['vol'] if df.iloc[0]['close'] > df.iloc[0]['open'] else -df.iloc[0]['vol']
    vol_ratio = vol / df['vol'].head(120).sum()
    hh = df['high'].head(120).max()
    ll = df['low'].head(120).min()
    price_ratio = (df.iloc[0]['close'] - ll)/(hh-ll)
    df = df.head(1)
    mini_df.loc[i, 'score'] = round(score*100,2)
    mini_df.loc[i, 'a_re'] = round(annualized_returns,2)
    mini_df.loc[i, 'r_sq'] = round(r_squared,2)
    mini_df.loc[i, 'vol_r'] = round(vol_ratio*100,3)
    mini_df.loc[i, 'p_r'] = round(price_ratio*100,2)

mini_df['rank'] = mini_df['circ_mv'].rank(method='average', ascending=False)
mini_df['rank_c'] = round(mini_df['rank'] / len(mini_df) *100,2)
mini_df['rank'] = mini_df['vol_r'].rank(method='average', ascending=True)
mini_df['rank_v'] = round(mini_df['rank'] / len(mini_df) *100,2)
mini_df['rank'] = mini_df['score'].rank(method='average', ascending=False)
mini_df['rank_s'] = round(mini_df['rank'] / len(mini_df) *100,2)
mini_df['rank'] = mini_df['dv_ttm'].rank(method='average', ascending=True)
mini_df['rank_dv'] = round(mini_df['rank'] / len(mini_df) *100,2)
mini_df['rank'] = mini_df['p_r'].rank(method='average', ascending=False)
mini_df['rank_p'] = round(mini_df['rank'] / len(mini_df) *100,2)


mini_df['rank'] = mini_df['rank_c'] + mini_df['rank_v'] + mini_df['rank_s'] + mini_df['rank_dv'] + mini_df['rank_p']

    

mini_df = mini_df.sort_values(by="rank", ascending=False)   
apply_info(mini_df) 

# mini_df = mini_df.sort_values(by='circ_mv')
# mini_df = mini_df.drop_duplicates(subset='industry', keep='first')
mini_df.to_csv("data/mini_df.csv")
mini_df[['ts_code', 'name','industry','act_ent_type','state','per','circ_mv','rank_c','score','rank_s','vol_r','rank_v','dv_ttm','rank_dv','p_r','rank_p','rank']].head(50)

3429


Processing: 100%|██████████| 3429/3429 [00:17<00:00, 201.14it/s]


,ts_code,name,industry,act_ent_type,state,per,circ_mv,rank_c,score,rank_s,vol_r,rank_v,dv_ttm,rank_dv,p_r,rank_p,rank
3699,301276.SZ,嘉曼服饰,服饰,民营企业,,,8.26,98.10,3.80,92.11,1.10,85.78,3.5431,91.03,57.74,79.50,446.52
788,001326.SZ,联域股份,电气设备,民营企业,,,7.27,98.83,6.42,89.85,1.22,87.93,4.2201,94.08,62.77,64.95,435.64
720,301151.SZ,冠龙节能,机械基件,NaN,,,9.37,97.01,33.29,74.66,0.99,82.71,3.5800,91.16,52.79,89.27,434.81
3209,603291.SH,联合水务,水务,外资企业,,,13.76,91.95,-0.03,98.26,0.75,70.90,1.7469,75.65,42.78,97.26,434.02
2017,301252.SZ,同星科技,机械基件,民营企业,,,9.02,97.39,2.19,93.82,1.58,92.32,1.8700,77.57,62.91,64.51,425.61
2250,301386.SZ,未来电器,电气设备,民营企业,,,9.64,96.69,3.74,92.24,1.03,84.15,3.5619,91.11,64.08,60.54,424.73
4059,603903.SH,中持股份,环境保护,无,,,14.16,91.43,13.87,85.01,1.01,83.39,2.7429,86.09,59.09,75.94,421.86
4990,301376.SZ,致欧科技,家居用品,民营企业,,,25.50,74.04,1.81,94.28,0.78,72.83,3.0015,88.16,50.56,92.18,421.49
509,001234.SZ,泰慕士,服饰,民营企业,hold,8.4,5.16,99.87,22.43,80.69,0.86,77.53,3.9362,92.68,60.89,70.63,421.40
1366,603075.SH,热威股份,家用电器,民营企业,,,7.76,98.60,13.30,85.48,0.71,67.88,2.5624,84.78,56.42,82.81,419.55


In [ ]:
pb_df = industry_df[industry_df["pb"] > 0]
print(len(pb_df))
pb_df = pb_df[pb_df['ts_code'].isin(up_list)]
print(len(pb_df))
pb_df.loc[:, 'circ_mv'] = round(pb_df['circ_mv'] / 10000,2)

for i in tqdm.tqdm(pb_df.index, desc="Processing"):
    ts_code = pb_df.loc[i, 'ts_code']
    df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
    score,annualized_returns,r_squared = apply_score(df,120)
    df = df.sort_values(by="trade_date", ascending=False)
    hh = df['high'].head(120).max()
    ll = df['low'].head(120).min()
    vol_ratio = df['vol'].head(20).sum() / df['vol'].head(120).sum()
    price_ratio = (df.iloc[0]['close'] - ll)/(hh-ll)
    df = df.head(1)
    pb_df.loc[i, 'score'] = round(score*100,2)
    pb_df.loc[i, 'a_re'] = round(annualized_returns,2)
    pb_df.loc[i, 'r_sq'] = round(r_squared,2)
    pb_df.loc[i, 'vol_r'] = round(vol_ratio*100,2)
    pb_df.loc[i, 'p_r'] = round(price_ratio*100,2)


pb_df['rank'] = pb_df['circ_mv'].rank(method='average', ascending=True)
pb_df['rank_c'] = round(pb_df['rank'] / len(pb_df) *100,2)
pb_df['rank'] = pb_df['pb'].rank(method='average', ascending=False)
pb_df['rank_pb'] = round(pb_df['rank'] / len(pb_df) *100,2)
pb_df['rank'] = pb_df['dv_ttm'].rank(method='average', ascending=True)
pb_df['rank_dv'] = round(pb_df['rank'] / len(pb_df) *100,2)
pb_df['rank'] = pb_df['score'].rank(method='average', ascending=False)
pb_df['rank_s'] = round(pb_df['rank'] / len(pb_df) *100,2)
pb_df['rank'] = pb_df['vol_r'].rank(method='average', ascending=False)
pb_df['rank_v'] = round(pb_df['rank'] / len(pb_df) *100,2)
pb_df['rank'] = pb_df['p_r'].rank(method='average', ascending=False)
pb_df['rank_p'] = round(pb_df['rank'] / len(pb_df) *100,2)

pb_df['rank'] = pb_df['rank_c'] + pb_df['rank_pb'] + pb_df['rank_dv'] + pb_df['rank_s'] + pb_df['rank_v'] +pb_df['rank_p']
pb_df['rank'] = pb_df['rank_pb'] + pb_df['rank_dv'] + pb_df['rank_s'] + pb_df['rank_v'] +pb_df['rank_p']


pb_df = pb_df.sort_values(by="rank", ascending=False)  
apply_info(pb_df)   
# pb_df = pb_df.drop_duplicates(subset='industry', keep='first')
pb_df.to_csv("data/pb_df.csv")

pb_df[['ts_code', 'name','industry','state','per','circ_mv','rank_c','pb','rank_pb','dv_ttm','rank_dv','score','rank_s','vol_r','rank_v','p_r', 'rank_p','rank']].head(50)

5042
3429


Processing:   0%|          | 0/3429 [00:00<?, ?it/s]

Processing: 100%|██████████| 3429/3429 [00:22<00:00, 153.08it/s]


,ts_code,name,industry,act_ent_type,state,per,circ_mv,rank_c,pb,rank_pb,dv_ttm,rank_dv,score,rank_s,vol_r,rank_v,p_r,rank_p,rank
861,601636.SH,旗滨集团,玻璃,NaN,,,160.47,87.55,1.2119,86.31,5.4659,97.43,0.66,95.57,1.30,93.99,43.67,96.92,470.22
4507,601107.SH,四川成渝,路桥,地方国企,sell,-2.7,111.60,80.90,0.9816,92.74,4.6512,95.71,0.43,95.98,1.57,86.57,44.44,96.40,467.40
2509,000157.SZ,中联重科,工程机械,NaN,hold,0.1,494.23,96.65,1.0621,90.23,4.5780,95.39,0.75,95.49,1.53,87.81,49.43,93.32,462.24
2869,601666.SH,平煤股份,煤炭开采,NaN,hold,-0.4,255.08,92.86,1.0411,90.81,9.4866,99.80,-0.01,98.05,1.65,83.97,53.32,88.48,461.11
2171,601857.SH,中国石油,石油开采,NaN,,,13779.57,99.94,1.0453,90.67,5.2879,97.08,-15.05,99.80,1.97,72.57,34.62,98.98,459.10
4590,002233.SZ,塔牌集团,水泥,NaN,sell,-2.7,90.55,75.94,0.7689,96.47,6.4603,98.43,7.33,89.30,2.00,71.19,42.32,97.40,452.79
4082,601163.SH,三角轮胎,汽车配件,NaN,,,122.48,82.62,0.9261,93.96,4.3762,94.63,7.84,88.95,1.77,79.82,47.93,94.65,452.01
3741,601101.SH,昊华能源,煤炭开采,NaN,hold,-6.1,122.11,82.50,1.0131,91.75,4.1274,93.73,0.43,95.98,2.05,69.63,24.71,99.74,450.83
2968,000598.SZ,兴蓉环境,水务,地方国企,,,214.48,91.16,1.2097,86.41,2.3583,83.38,-2.82,99.24,1.52,88.15,50.19,92.58,449.76
1932,000028.SZ,国药一致,医药商业,NaN,hold,0.4,143.94,85.59,0.9258,93.99,2.2584,82.18,-0.06,98.40,1.48,89.87,55.34,84.75,449.19


In [ ]:
buy_df = pd.DataFrame()

for b in buy_list:
    # 从 daily 里读 csv,只读第一行然后按 circ_mv 排序
    df = pd.read_csv(os.path.join(daily_data_dir, f"{b}.csv"))
    df = apply_bbands(df,period=60)
    
    df = df.sort_values(by="trade_date", ascending=False)
    df = df.head(1)
    ts_code = df['ts_code'].iloc[0]
    name = chnname(ts_code)
    industry = chnindustry(ts_code)
    df['name'] = name
    df['industry'] = industry
    buy_df = pd.concat([buy_df, df])

buy_df = buy_df.sort_values(by="rel", ascending=True)    
buy_df['circ_mv'] = buy_df['circ_mv'] / 1e4
buy_df.reset_index(inplace=True)
apply_info(buy_df)
buy_df.to_csv("data/buy_df.csv")
print(len(buy_df))
buy_df[['ts_code', 'name','industry','state','per','circ_mv','rel']].head(50)

In [19]:
up_df = pd.DataFrame()

for b in tqdm.tqdm(up_list, desc="Processing"):
    # 从 daily 里读 csv,只读第一行然后按 circ_mv 排序
    df = pd.read_csv(os.path.join(daily_data_dir, f"{b}.csv"))
    score,annualized_returns,r_squared = apply_score(df,25)
    df = df.sort_values(by="trade_date", ascending=False)
    vol_ratio = df['vol'].head(5).sum() / df['vol'].head(120).sum()
    df = df.head(1)
    ts_code = df['ts_code'].iloc[0]
    name = chnname(ts_code)
    industry = chnindustry(ts_code)
    df['name'] = name
    df['industry'] = industry
    df['vol_r'] = round(vol_ratio*100,2)
    df['score'] = round(score*100,2)
    df['a_re'] = round(annualized_returns,2)
    df['r_sq'] = round(r_squared,2)
    df['vol_r'] = round(vol_ratio*100,2)
    up_df = pd.concat([up_df, df])

up_df['rank'] = up_df['score'].rank(method='average', ascending=False)
up_df['rank_s'] = round(up_df['rank'] / len(up_df) *100,2)
up_df['rank'] = up_df['vol_r'].rank(method='average', ascending=True)
up_df['rank_v'] = round(up_df['rank'] / len(up_df) *100,2)
up_df['rank'] = up_df['rank_s'] + up_df['rank_v']

print(len(up_df))
up_df = up_df.sort_values(by="rank", ascending=False)    
up_df['circ_mv'] = round(up_df['circ_mv'] / 1e4,2)
up_df.reset_index(inplace=True)
apply_info(up_df)
up_df.to_csv("data/up_df.csv")
up_df[['ts_code', 'name', 'industry','state','per','circ_mv','score','rank_s','vol_r','rank_v','rank']].head(50)

Processing: 100%|██████████| 3875/3875 [00:27<00:00, 139.52it/s]


3875


,ts_code,name,industry,state,per,circ_mv,score,rank_s,vol_r,rank_v,rank
0,02402.HK,亿华通,NaN,,,0.00,-2.56,93.43,47.04,99.92,193.35
1,02157.HK,乐普生物－Ｂ,NaN,,,0.00,-19.71,98.30,24.80,94.72,193.02
2,688656.SH,浩欧博,医疗保健,,,65.67,-45.72,99.74,20.67,90.27,190.01
3,09880.HK,优必选,NaN,,,0.00,-1.76,92.57,24.72,94.65,187.22
4,02228.HK,QUANTUMPH-P,NaN,,,0.00,0.72,80.21,60.27,100.00,180.21
5,600225.SH,*ST卓朗,软件服务,,,51.81,-56.60,99.92,15.83,78.65,178.57
6,600826.SH,兰生股份,文教休闲,,,73.26,-0.00,85.90,20.49,89.91,175.81
7,02291.HK,心泰医疗,NaN,,,0.00,-0.03,87.77,19.20,87.25,175.02
8,002402.SZ,和而泰,元器件,,,178.71,2.88,76.03,25.49,95.28,171.31
9,603039.SH,泛微网络,软件服务,,,132.88,-1.85,92.67,15.47,77.12,169.79


In [ ]:
f_df = pd.DataFrame()

for c in fetf_df['ts_code']:
    df = pd.read_csv(os.path.join(daily_data_dir, f"{c}.csv"))
    df = apply_bbands(df,period=60)
    df = df.sort_values(by="trade_date", ascending=False)
    df = df.head(1)
    df['name'] = chnname(c)
    f_df = pd.concat([f_df, df], ignore_index=True)
  
apply_info(f_df)
f_df = f_df.sort_values(by="rel", ascending=False)
f_df.reset_index(inplace=True)


money = 900000
f = money* 0.5
print(f,f/10)
f3 = money*0.5/3
print(f3)
f_df[['ts_code', 'name','rel','state','per']]
